In [1]:
#Implementation of Shamir Secret Sharing Scheme 

import random 
from math import ceil 
from decimal import *


global field_size 
field_size = 10**5

def reconstructSecret(shares): 

	# Combines shares using 
	# Lagranges interpolation. 
	# Shares is an array of shares 
	# being combined 
	sums, prod_arr = 0, [] 
	
	for j in range(len(shares)): 
		xj, yj = shares[j][0],shares[j][1] 
		prod = Decimal(1) 

		for i in range(len(shares)): 
			xi = shares[i][0] 
			if i != j: prod *= Decimal(Decimal(xi)/(xi-xj)) 
				
		prod *= yj 
		sums += Decimal(prod) 
		
	return int(round(Decimal(sums),0)) 

def polynom(x,coeff): 

	# Evaluates a polynomial in x 
	# with coeff being the coefficient 
	# list 
	return sum([x**(len(coeff)-i-1) * coeff[i] for i in range(len(coeff))]) 

def coeff(t,secret): 

	# Randomly generate a coefficient 
	# array for a polynomial with 
	# degree t-1 whose constant = secret''' 
	coeff = [random.randrange(0, field_size) for _ in range(t-1)] 
	coeff.append(secret) 
	
	return coeff 

def generateShares(n,m,secret): 

	# Split secret using SSS into 
	# n shares with threshold m 
	cfs = coeff(m,secret) 
	shares = [] 

	for i in range(1,n+1): 
		r = random.randrange(1, field_size) 
		shares.append([r, polynom(r,cfs)]) 
	
	return shares 


# Driver code 
if __name__ == '__main__': 

	# (3,5) sharing scheme 
	t,n = 3, 5
	secret = 1234
	print('Original Secret:', secret) 

	# Phase I: Generation of shares 
	shares = generateShares(n, t, secret) 
	print('\nShares:', *shares) 

	# Phase II: Secret Reconstruction 
	# Picking t shares randomly for 
	# reconstruction 
	pool = random.sample(shares, t) 
	print('\nCombining shares:', *pool) 
	print("Reconstructed secret:", reconstructSecret(pool)) 



Original Secret: 1234

Shares: [73426, 379174093214594] [98082, 676575735132322] [10410, 7621973980594] [95826, 645809740568194] [11357, 9071740588497]

Combining shares: [11357, 9071740588497] [10410, 7621973980594] [73426, 379174093214594]
Reconstructed secret: 1234


## Ramp Secret Sharing

In [2]:
import rs_sss
import numpy as np

In [3]:
deg = 8 # m of GF(2^m)
s = rs_sss.RS_SSS(deg)

In [4]:
threshold = 8 # k
ramp = 3  # L
num = 11  # n
s.initialize(threshold, ramp, num)

In [5]:
# generate a random secret of 1024 bytes (deg = 8)
orig_size = 1024
orig_secret = np.random.randint(0, (1 << deg) - 1, orig_size) 

# set the secret into the instance
s.set_secret(orig_secret) 

In [6]:
# generate shares from the given secret
s.generate_shares()

# shares are generated in SSS._shares[]
for i in range(num):
    print("Share {0}: {1}".format(i, s.get_shares()[i]))

Share 0: [207 181 140  12 217  21  19  59  84 133 209  41 220  86 253 227   6  70
 108  13 215  70 232  57  17  44 143  94 159   7 143  83 104  10 162 225
 245  32 222 159  44 140  53  94 230  62 127 187 134 135 169  59 242 108
 177 213  52  62  11  67 128 150  78 227 131  29 191  64 183 155 240 163
 128  45 144 216 216 233  28  65 206 198 157  39  85  93   7  77  19 231
  64 207  73  98 159 231 216  38  68 248  91 197 125  27  52  42 157 188
 213 189 110  42 112  66  32  79 189 160  81 177  35  56 234  59  62  35
  46  31 179  98 199 204 208 112 157 235 100 138  65  26 235 123 209 240
 163  76  91  73  24  24 138  73 224 132 184 221  13 210  20  71 165 168
  93  21 172  74 244 154  37  28  47 237 140 128  14   0  22 148 242 162
  31 142 207  73 111  78 104 116  13  24  79 213 158 139 198 242 234  39
  56 239 207  82  57 144 231   6 133 123 251  33 247  36 158  87 210  46
  35 102 227  64 230  80 200   3  27  87  38 120 153  30 113  37 203  46
  21 233  16   9   0 140 120 185  23 107 1

In [7]:
shares = []  # list of shares for secret reconstruction
index_list = [0, 1, 2, 8, 4, 7, 9, 10]  # list of share indices for secret reconstruction

# copy from the instance variable
for i in index_list:
    shares.append(s.get_shares()[i])
    
# initialize again and remove all data from the instance
s.__init__(deg)
s.initialize(threshold, ramp, num)

# set copied shares and their indices to the instance
s.set_external_shares(shares, index_list)

In [8]:
# reconstruct the secret
s.reconstruct_secret(orig_size)
reco_secret = s.get_secret()

# check if the original secret coincides with the reconstructed one
print("Original secret == Reconstructed secret ?: {0}".format(np.allclose(orig_secret, reco_secret)))

Original secret == Reconstructed secret ?: True


## Shamir Secret Sharing 

In [9]:
from Crypto.Protocol.SecretSharing import Shamir
from Crypto.Random import get_random_bytes
from binascii import hexlify
# Generating Key
key = get_random_bytes(16)
print("This is the Original Key: ", key)

# Sharing the key by spliting into 5 keys, we need 2 keys to get the original key. 
shares = Shamir.split(2, 5, key)
#for idx, share in shares:
    #print ("Index #%d: %s" % (idx, hexlify(share)))
    
shares1 = []
for i in shares:
    shares1.append(i)
    
# Combining 2 keys to get the original key
Shamir.combine([shares1[0], shares1[1]])
combined = Shamir.combine([shares1[0], shares1[1]])
print("This is the combined key: ", combined)

# Checking both keys are a match
print("True: Keys are a match, False: Keys are different")
print("If both keys are a match:")
print(combined==key)

This is the Original Key:  b':(\xd8\xdd\x00-\x18\xb1\x8eY\xb71\xf1n\x1a\x0c'
This is the combined key:  b':(\xd8\xdd\x00-\x18\xb1\x8eY\xb71\xf1n\x1a\x0c'
True: Keys are a match, False: Keys are different
If both keys are a match:
True


## Ramp Secret Sharing 

In [10]:
# Get the key
key = b'~s]m\xa8\xd6\xca8\x13\xce\x96\xca\x8e\xa7\x105\xd5\x80\xb4\x1fj\xd5\x0e5<AH\x8a\nU)\x18'

# Convert key to String
#t = key.decode('utf-8','ignore')


#### Trying to implment Ramp Secret Sharing Technique

In [11]:
# generate a random secret of 1024 bytes (deg = 8)
orig_size = 1024
orig_secret = np.random.randint(0, (1 << deg) - 1, orig_size)
print(orig_secret)
# set the secret into the instance
s.set_secret(orig_secret) 

[173 191 190 ...  13 252  15]


### Generating Shares from Secret Sharing 

In [12]:
# generate shares from the given secret
s.generate_shares()

# shares are generated in SSS._shares[]
for i in range(num):
    print("Share {0}: {1}".format(i, s.get_shares()[i]))

Share 0: [ 51 127  17  88  37 221 196 218 189 255 193 194 220  81 114  66 105  49
 134 254 174  45  31  88  98 234  90 179  26 206 200 103 178  50  71  12
  65  78 152  39 184 147  52  77  62  48   6 185 186  81 134 237 125  81
 104  99 198 185  29 103 255 190 159 162 217 212  77 109  39 173 254 129
 202 128 247  57 104   2  97 180  19 175  91 253 158  39  13 203 161 189
 195  22   9 154 186 152  66 107 139 229  97 246  44  55 123  44   9 215
  54  90 170 246 179 178  63 233 238 227 252  12  38  24 238  51   2 251
  26 144  14 153 131  33 104 239 239 157 178 221  71  98 178 176  63 217
  31 164 219 188  83   9  93  88 106 216 145  81  12   9 245 180  56  26
 244  64  24 105  73  33 173 139 161   3  19 233 225  16 145  42 104  56
 170  65  37  85 129  55 193  75 117 237 177 207 166  27 148  64  38 166
 100 174  51 112  14  82 183 200 144  64 105   4   9   0 136  33 243 116
 152 166 179 251 245  49   5 170 245  53  36 147 176 115  96 124 240  87
  79  86  60 146  97   8 248 255  25 165 2

### Another Implementation for Secret Sharing Scheme

#### Ramp Secret Sharing 

In [13]:
import random

In [14]:
N = 20
T = 8
K = 2
Q = 41
assert(T+K <= N)

In [15]:
SECRET_POINTS =     [ -p % Q for p in range(1, K+1) ]
RANDOMNESS_POINTS = [ -p % Q for p in range(K+1, K+T+1) ]
assert(set(SECRET_POINTS).intersection(RANDOMNESS_POINTS) == set())

def sample_packed_polynomial(secrets):
    assert(len(secrets) == K)
    points = SECRET_POINTS + RANDOMNESS_POINTS
    values = secrets + [ random.randrange(Q) for _ in range(T) ]
    return list(zip(points, values))

In [16]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

# from http://www.ucl.ac.uk/~ucahcjm/combopt/ext_gcd_python_programs.pdf
def egcd_binary(a,b):
    u, v, s, t, r = 1, 0, 0, 1, 0
    while (a % 2 == 0) and (b % 2 == 0):
        a, b, r = a//2, b//2, r+1
    alpha, beta = a, b
    while (a % 2 == 0):
        a = a//2
        if (u % 2 == 0) and (v % 2 == 0):
            u, v = u//2, v//2
        else:
            u, v = (u + beta)//2, (v - alpha)//2
    while a != b:
        if (b % 2 == 0):
            b = b//2
            if (s % 2 == 0) and (t % 2 == 0):
                s, t = s//2, t//2
            else:
                s, t = (s + beta)//2, (t - alpha)//2
        elif b < a:
            a, b, u, v, s, t = b, a, s, t, u, v
        else:
            b, s, t = b - a, s - u, t - v
    return (2 ** r) * a, s, t


def inverse(a):
    _, b, _ = egcd_binary(a, Q)
    return b

In [17]:
def lagrange_constants_for_point(points, point):
    constants = [0] * len(points)
    for i in range(len(points)):
        xi = points[i]
        num = 1
        denum = 1
        for j in range(len(points)):
            if j != i:
                xj = points[j]
                num = (num * (xj - point)) % Q
                denum = (denum * (xj - xi)) % Q
        constants[i] = (num * inverse(denum)) % Q
    return constants


def interpolate_at_point(points_values, point):
    points, values = zip(*points_values)
    constants = lagrange_constants_for_point(points, point)
    return sum( vi * ci for vi, ci in zip(values, constants) ) % Q

In [24]:
SHARE_POINTS = [ p for p in range(1, N+1) ]
assert(set(SHARE_POINTS).intersection(SECRET_POINTS) == set())
assert(set(SHARE_POINTS).intersection(RANDOMNESS_POINTS) == set())

def packed_share(secrets):
    polynomial = sample_packed_polynomial(secrets)
    shares = [ interpolate_at_point(polynomial, p) for p in SHARE_POINTS ]
    return shares

def packed_reconstruct(shares):
    points = SHARE_POINTS
    values = shares
    polynomial = [ (p,v) for p,v in zip(points, values) if v is not None ]
    secrets = [ interpolate_at_point(polynomial, p) for p in SECRET_POINTS ]
    return secrets

secrets = [5,6]
shares = packed_share(secrets)
for i in range(N-(T+K)):
    shares[i] = None
#shares[-1] = None  # would fail; we need T+K points to reconstruct
reconstructed_secrets = packed_reconstruct(shares)
assert(reconstructed_secrets == secrets)